In [1]:
import pandas as pd
import json
import numpy as np

Read the jsonl file.

In [2]:
def open_jsonl(filename):
    sentences = []
    with open(filename,  encoding="utf8") as json_file:
        json_list = list(json_file)

    for json_str in json_list:
        result = json.loads(json_str)
        sentences.append(result)
    
    return pd.DataFrame(sentences)

In [3]:
train = open_jsonl('../train.jsonl')
test = open_jsonl('../test.jsonl')

separate data between the text sentences and the label

In [14]:
X = train['text']
y = train['lang']

X_train = X
y_train = y

split data

In [5]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

convert text to token using CountVectorizer

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
# cv = CountVectorizer(ngram_range=(1, 3))
X_train_counts = cv.fit_transform(X_train)

In [7]:
# pd.DataFrame([cv.vocabulary_])

calculate the TF-IDF

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_counts)

train the model (Multinomial Naive Bayes)

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [17]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)
# clf = make_pipeline(StandardScaler(with_mean = False), SVC(gamma='auto'))
# clf.fit(X_train_tfidf, y_train)


### Testing using train_test_split

In [18]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train_tfidf, y_train, cv=5, scoring='f1_macro')


In [19]:
scores

array([0.98885772, 0.9886649 , 0.98846945, 0.98959646, 0.98944482])

In [34]:
test_sentences = X_test
X_new_counts = cv.transform(test_sentences)
X_new_tfidf = tfidf.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted, target_names=np.unique(y_train)))

              precision    recall  f1-score   support

          af       0.99      1.00      1.00      3936
          az       0.99      1.00      0.99      3933
          bg       1.00      1.00      1.00      4016
          cs       0.99      1.00      1.00      3995
          da       0.99      1.00      1.00      3983
          de       0.99      1.00      1.00      3977
          el       1.00      1.00      1.00      3979
          en       0.89      1.00      0.94      4020
          es       0.99      1.00      1.00      3977
          fi       0.99      1.00      0.99      4039
          fr       0.86      1.00      0.92      3927
          hr       1.00      1.00      1.00      4028
          it       1.00      1.00      1.00      4055
          ko       1.00      1.00      1.00      4036
          nl       1.00      1.00      1.00      3970
          no       1.00      1.00      1.00      4016
          pl       1.00      1.00      1.00      3968
          ru       1.00    

### Testing using the test.jsonl

In [20]:
X_test = test['text']

X_new_counts = cv.transform(X_test)
X_new_tfidf = tfidf.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)


In [21]:
test.head()

,id,text
0,0,Численность населения агломерации Парижа в 201...
1,1,Наиболее выдающийся вклад в создание современн...
2,2,С началом франко-прусской войны 1870—1871 годо...
3,3,"Основное преимущество офсетных антенн в том, ч..."
4,4,"Выручка от реализации продукции, товаров, рабо..."


In [22]:
# df_new = pd.concat([test.text, pd.DataFrame(predicted, columns=['predicted'])], axis=1)
df_new = pd.concat([test.id, pd.DataFrame(predicted, columns=['lang'])], axis=1)

df_new

,id,lang
0,0,ru
1,1,ru
2,2,ru
3,3,ru
4,4,ru
...,...,...
99133,99133,da
99134,99134,da
99135,99135,da
99136,99136,da


In [25]:
df_new.to_json('predictions.jsonl', orient='records')

In [ ]:
df_new.shape
df_new[df_new.predicted=='zh']
weird = df_new.iloc[76104].text.split(', ')
weird

['Orbitoproetus',
 'Ormistoniella',
 'Osmolskia',
 'Paladin',
 'Palpebralina',
 'Panibole',
 'Paraglobusia',
 'Paraproetus',
 'Parvidumus',
 'Pedinocoryphe',
 'Pedinodechenella',
 'Perexigupyge',
 'Phillibole',
 'Phillibolina',
 'Philliboloides',
 'Phillipsia',
 'Piltonia',
 'Plesiowensus',
 'Podoliproetus',
 'Praedechenella',
 'Pragoproetus',
 'Prantlia',
 'Proetidellinae',
 'Proetiella',
 'Proetinae',
 'Proetus',
 'Pseudodechenella',
 'Pseudodudu',
 'Pseudoproetus',
 'Pseudospatulina',
 'Pseudowaribole',
 'Pudoproetus',
 'Reediella',
 'Rhenogriffides',
 'Richterella',
 'Ryckholtia',
 'Schaderthalaspis',
 'Schizoproetina',
 'Schizoproetoides',
 'Schizoproetus',
 'Semiproetus',
 'Sevillia',
 'Silesiops',
 'Simaproetus',
 'Spinibole',
 'Spinibolops',
 'Tawstockia',
 'Tetinia',
 'Thebanaspis',
 'Thigriffides',
 'Tropidocare',
 'Tropidocoryphinae',
 'Typhloproetus',
 'Unguliproetus',
 'Warburgellinae',
 'Waribole',
 'Weberides',
 'Weyeraspis',
 'Winiskia',
 'Winterbergia',
 'Witryides',
 

In [ ]:
for w in weird:
    if train[train.text.str.contains(w)].shape[0] > 0:
        print(w)

Paladin
Sevillia


In [ ]:
train[train.text.str.contains('Paladin')]

,id,lang,text
70516,70516,el,Το όγδοο άλμπουμ των HammerFall που κυκλοφόρησ...
92564,92564,es,"Gastón II, apodado el Paladino, fue conde de F..."
119396,119396,zh,"作為作品的主要敵人,被描述為最弱小的怪物,擁有孩童般的智力與力量,但懂得使用武器與集團行動的..."
147223,147223,it,L'edizione italiana è curata da Ludovica Bonan...
149073,149073,it,Nel 1351 Raffaele Maremonti ebbe l'investitura...
223813,223813,no,"I 1852 ble jernbanen ""Compagnies ferroviaires ..."
256137,256137,de,"""Hero's Quest"" bzw. ""Quest for Glory I"" wird ü..."
297515,297515,en,During the late 1970s in the United States pai...
319911,319911,hr,Videospot za pjesmu je objavljen 23. svibnja 2...
339516,339516,fr,La seule exploitation minière à grande échelle...


In [ ]:
train.iloc[119396].text

'作為作品的主要敵人,被描述為最弱小的怪物,擁有孩童般的智力與力量,但懂得使用武器與集團行動的特性讓牠們在狹小場所如洞窟內深具威脅,是新人冒險者的主要對手。擁有較高的資質、長期累積經驗並且生存的哥布林,能夠成長為不同的亞種:鄉巴佬(Hob)、薩滿(Shaman)、英雄(Champion)、王(Lord),甚至是聖騎士(Paladin)與龍騎士(Dragoon)。作品中並未出現過哥布林的母體,哥布林透過搶奪類人生物(人類、精靈、矮人、半身人等)的女性為牠們延續後代。'